## Import data

In [1]:
!pip install kaggle
!pip install autocorrect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 kB 14.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622364 sha256=d19aebde56df8b87428665aff167411c1e95241447ba5ad04207913811225062
  Stored in directory: /root/.cache/pip/wheels/b5/7b/6d/b76b29ce11ff8e2521c8c7dd0e5bfee4fb1789d76193124343
Successfully built autocorrect


In [2]:
from google.colab import files
files.upload()  # Upload kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"odelepang","key":"fc5a9458eb962e592ab2d28403bb8685"}'}

In [3]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d danofer/sarcasm
!unzip sarcasm.zip

Dataset URL: https://www.kaggle.com/datasets/danofer/sarcasm
License(s): copyright-authors
 94% 203M/216M [00:01<00:00, 130MB/s]
100% 216M/216M [00:01<00:00, 122MB/s]
Archive:  sarcasm.zip
  inflating: test-balanced.csv       
  inflating: test-unbalanced.csv     
  inflating: train-balanced-sarc.csv.gz  
  inflating: train-balanced-sarcasm.csv  


In [5]:
import pandas as pd
import numpy as np
import spacy
import re
import string
from spacy.lang.en.stop_words import STOP_WORDS as stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack, csr_matrix
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from nltk import bigrams
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures


# from autocorrect import Speller

In [6]:
df = pd.read_csv('train-balanced-sarcasm.csv')
df.head()

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...


## Data cleaning

In [7]:
#Remove all na rows
df = df.dropna()

In [8]:
#Selecting only the comment and label column
filtered_df = df[df['subreddit'].isin(['news', 'politics', 'worldnews'])]
comments = filtered_df.loc[:,["label", "comment", "parent_comment"]]

In [9]:
#Word count column
comments["word_count"] = comments["comment"].apply(lambda x: len(x.split()))

In [10]:
#Capital letter counts
def count_cap(x):
  result = 0
  for c in x:
    if c.isupper():
      result += 1
  return result

comments["capital_count"] = comments["comment"].apply(lambda x: count_cap(x))

In [11]:
#Punctuation count
comments["punc_count"] = comments["comment"].apply(lambda x: sum(1 for i in x if i in string.punctuation))

In [12]:
#Lower case
comments["comment"] = comments["comment"].apply(lambda x: x.lower())
comments["parent_comment"] = comments["parent_comment"].apply(lambda x: x.lower())

In [13]:
#List of contractions manually modified for convenience
contractions = {
"i ain't": "i am not",
"you ain't": "you are not",
"he ain't": "he is not",
"she ain't": "she is not",
"they ain't": "they are not",
"it ain't": "it is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": " what is",
"what've": "what have",
"when's": " when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}


In [14]:
#Expand words
def decontract(words):
  final = []
  for word in words:
    if word in contractions:
      final.append(contractions[word])
    else:
      final.append(word)
  return " ".join(final)

comments["comment"] = comments["comment"].apply(lambda x: decontract(x.split()))
comments["comment"] = comments["comment"].apply(lambda x: re.sub(r"\'s", " is", x))
comments["parent_comment"] = comments["parent_comment"].apply(lambda x: decontract(x.split()))
comments["parent_comment"] = comments["parent_comment"].apply(lambda x: re.sub(r"\'s", " is", x))

In [15]:
#Lemmatize
nlp = spacy.load("en_core_web_sm")
remove_pos = ['PUNCT', 'SPACE', 'NUM', 'DET', 'ADP', 'PRON']
remove = ['to', 'be']
def make_to_base(x):
  x = str(x)
  x_list = []
  doc = nlp(x)
  lemma = [token.lemma_.lower() for token in doc if token.lemma_ not in remove and token.pos_ not in remove_pos and token.is_alpha]
  return " ".join(lemma)

comments["comment"] = comments["comment"].apply(lambda x: make_to_base(x))
comments["parent_comment"] = comments["parent_comment"].apply(lambda x: make_to_base(x))
comments.to_csv('/content/drive/My Drive/lemmatised.csv', index=False)

KeyboardInterrupt: 

In [17]:
from google.colab import drive
drive.mount('/content/drive')

comments = pd.read_csv('/content/drive/My Drive/lemmatised.csv')
comments = comments.dropna()

Mounted at /content/drive


In [18]:
comments.insert(0, 'comment_id', range(1, len(comments) + 1))
X = comments.drop(['label'], axis = 1)
y = comments[['comment_id', 'label']]
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size = 0.2, random_state = 123)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size = 0.2, random_state = 123)

In [32]:
#Edit to change to top x bigrams
bigram_vectorizer = TfidfVectorizer(ngram_range=(2, 2), min_df = 2)
bigrams = bigram_vectorizer.fit_transform(X_train["comment"])
bigram_df = pd.DataFrame(bigrams.toarray(), columns=bigram_vectorizer.get_feature_names_out())
top_50_bigrams = bigram_df.sum().nlargest(50).index

for bigram in top_50_bigrams:
  comments['comment'] = comments['comment'].str.replace(bigram, bigram.replace(" ", "_"))

KeyboardInterrupt: 

In [20]:
def remove_stopwords(words):
  final = []
  for word in words:
    if word not in stopwords:
      final.append(word)
  return " ".join(final)

comments["comment"] = comments["comment"].apply(lambda x: remove_stopwords(x.split()))
comments["parent_comment"] = comments["parent_comment"].apply(lambda x: remove_stopwords(x.split()))

In [21]:
from textblob import TextBlob
from textblob.sentiments import PatternAnalyzer

def get_polarity(x):
  return TextBlob(x, analyzer=PatternAnalyzer()).sentiment.polarity

def get_subjectivity(x):
  return TextBlob(x, analyzer=PatternAnalyzer()).sentiment.subjectivity

comments["comment_polarity"] = comments["comment"].apply(lambda x: get_polarity(x))
comments["comment_subjectivity"] = comments["comment"].apply(lambda x: get_subjectivity(x))
comments["parent_polarity"] = comments["parent_comment"].apply(lambda x: get_polarity(x))
comments["parent_subjectivity"] = comments["parent_comment"].apply(lambda x: get_subjectivity(x))

## Modelling

In [31]:
comments = comments.loc[(comments['comment'].apply(lambda x: len(x) != 0)) & (comments['parent_comment'].apply(lambda x: len(x) != 0))]

X_train_bigram = comments.loc[comments['comment_id'].isin(X_train['comment_id'])]
X_val_bigram = comments.loc[comments['comment_id'].isin(X_val['comment_id'])]
X_test_bigram = comments.loc[comments['comment_id'].isin(X_test['comment_id'])]
y_train_bigram = y_train.loc[y_train['comment_id'].isin(comments['comment_id']), 'label']
y_val_bigram = y_val.loc[y_val['comment_id'].isin(comments['comment_id']), 'label']
y_test_bigram = y_test.loc[y_test['comment_id'].isin(comments['comment_id']), 'label']